In [2]:
import cv2
import numpy as np
import tkinter as tk
import tkinter.font as tkFont
from tkinter import filedialog
from PIL import Image, ImageTk
from tkinter import PhotoImage
import tkinter.messagebox as messagebox
import shutil
import os
import subprocess
import time

class ComputerVisionApp:
    def __init__(self, master):
        self.master = master
        self.master.title("UAS Computer Vision App - Realtime Emotions Detection")
        
        #  check folder
        self.check_folder()
        # Create a frame as a container for the buttons
        self.button_frame = tk.Frame(self.master)
        self.button_frame.pack(pady=200) 
        # Load the image
        self.image = Image.open('./image/luffy.png')  
        self.photo = ImageTk.PhotoImage(self.image)

        # Create a label to display the image
        self.image_label = tk.Label(self.master, image=self.photo)
        self.image_label.place(relx=1, rely=1, anchor='se') 
       
        
    
        self.load_icon = PhotoImage(file='./image/icon.png')  
        self.quit_icon = PhotoImage(file='./image/quit.png')
        self.eye_icon = PhotoImage(file='./image/eye.png')
        
        self.load_button = tk.Button( self.button_frame, text="Train Data", command=self.open_train_data,  compound=tk.TOP, image=self.load_icon, padx=10, pady=5)
        self.load_button.pack(side=tk.LEFT,padx=20)

        self.process_button = tk.Button( self.button_frame, text="Recognition", command=self.process_image,  compound=tk.TOP, image=self.eye_icon, padx=10, pady=5)
        self.process_button.pack(side=tk.LEFT,padx=20)

        self.quit_button = tk.Button( self.button_frame, text="Quit", command=self.master.destroy,  compound=tk.TOP, image=self.quit_icon, padx=10, pady=5)
        self.quit_button.pack(side=tk.LEFT,padx=20)

        # Initialize variables
        self.image_path = None
        self.cv_image = None

    def load_image(self):
        self.image_path = filedialog.askopenfilename()
        if self.image_path:
            self.cv_image = cv2.imread(self.image_path)
            self.show_image(self.cv_image)

    def process_image(self):
        !python ./src/emotions.py --mode display
    

    def show_image(self, image):
        # Convert the OpenCV image to a format that Tkinter can display
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageTk.PhotoImage(image)

        # Update the image label
        self.image_label.configure(image=image, compound=tk.BOTTOM, text="Your text here")
        self.image_label.image = image
        
    def open_train_data(self):
        self.master.withdraw()  # Hide the main window

        self.new_window = tk.Toplevel(self.master)
        self.new_window.title("Train Data")
        self.new_window.geometry("600x500")
        self.new_window.protocol("WM_DELETE_WINDOW", lambda: self.on_close_train_window())  # Event when new window is closed

        # Add an image to the train window
        train_image = Image.open('./image/zoro.png')
        train_photo = ImageTk.PhotoImage(train_image)

        train_image_label = tk.Label(self.new_window, image=train_photo)
        train_image_label.image = train_photo
        train_image_label.place(relx=1, rely=1, anchor='se')

#         button select file
        button_file=tk.Button(self.new_window)
        button_file["bg"] = "#33eb0e"
        ft = tkFont.Font(family='Times',size=10)
        button_file["font"] = ft
        button_file["fg"] = "#000000"
        button_file["justify"] = "center"
        button_file["text"] = "Select CSV File"
        button_file.place(x=70,y=50,width=100,height=25)
        button_file["command"] = self.select_file
        
        self.entry_epoch=tk.Entry(self.new_window)
        self.entry_epoch["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        self.entry_epoch["font"] = ft
        self.entry_epoch["fg"] = "#333333"
        self.entry_epoch["justify"] = "center"
        self.entry_epoch["text"] = "Entry"
        self.entry_epoch.place(x=160,y=110,width=70,height=25)
        # Validate batch size entry
        self.entry_epoch.config(validate="key")
        self.entry_epoch['validatecommand'] = (self.entry_epoch.register(self.validate_batch_size), '%P')
        
        label_epoch=tk.Label(self.new_window)
        ft = tkFont.Font(family='Times',size=10)
        label_epoch["font"] = ft
        label_epoch["fg"] = "#333333"
        label_epoch["justify"] = "center"
        label_epoch["text"] = "Epoch"
        label_epoch.place(x=60,y=110,width=70,height=25)

        label_batch=tk.Label(self.new_window)
        ft = tkFont.Font(family='Times',size=10)
        label_batch["font"] = ft
        label_batch["fg"] = "#333333"
        label_batch["justify"] = "center"
        label_batch["text"] = "batch size"
        label_batch.place(x=300,y=110,width=70,height=25)

        self.entry_batch=tk.Entry(self.new_window)
        self.entry_batch["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        self.entry_batch["font"] = ft
        self.entry_batch["fg"] = "#333333"
        self.entry_batch["justify"] = "center"
        self.entry_batch["text"] = "Entry Batch Size"
        self.entry_batch.place(x=430,y=110,width=70,height=25)
        # Validate batch size entry
        self.entry_batch.config(validate="key")
        self.entry_batch['validatecommand'] = (self.entry_batch.register(self.validate_batch_size), '%P')
        
        
        self.file_label=tk.Label(self.new_window)
        ft = tkFont.Font(family='Times',size=10)
        self.file_label["font"] = ft
        self.file_label["fg"] = "#333333"
        self.file_label["justify"] = "center"
        self.file_label["text"] = "file not selected"
        self.file_label.place(x=200,y=50,width=300,height=25)

        btn_train=tk.Button(self.new_window)
        btn_train["bg"] = "#0eebe4"
        ft = tkFont.Font(family='Times',size=10)
        btn_train["font"] = ft
        btn_train["fg"] = "#000000"
        btn_train["justify"] = "center"
        btn_train["text"] = "Train Data"
        btn_train.place(x=270,y=190,width=70,height=25)
        btn_train["command"] = self.train_data

        btn_default=tk.Button(self.new_window)
        btn_default["bg"] = "#ffd700"
        ft = tkFont.Font(family='Times',size=10)
        btn_default["font"] = ft
        btn_default["fg"] = "#000000"
        btn_default["justify"] = "center"
        btn_default["text"] = "Use Default Model"
        btn_default.place(x=50,y=310,width=120,height=25)
        btn_default["command"] = self.model_default

        btn_back=tk.Button(self.new_window)
        btn_back["bg"] = "#ff5722"
        ft = tkFont.Font(family='Times',size=10)
        btn_back["font"] = ft
        btn_back["fg"] = "#000000"
        btn_back["justify"] = "center"
        btn_back["text"] = "Back"
        btn_back.place(x=60,y=410,width=70,height=25)
        btn_back["command"] = lambda: self.back_to_main(self.new_window)
        
        btn_extract=tk.Button(self.new_window)
        btn_extract["bg"] = "#00babd"
        ft = tkFont.Font(family='Times',size=10)
        btn_extract["font"] = ft
        btn_extract["fg"] = "#000000"
        btn_extract["justify"] = "center"
        btn_extract["text"] = "Extract Image"
        btn_extract.place(x=430,y=50,width=100,height=25)
        btn_extract["command"] = self.extract_image

        self.loading_label=tk.Label(self.new_window)
        ft = tkFont.Font(family='Times',size=10)
        self.loading_label["font"] = ft
        self.loading_label["fg"] = "#333333"
        self.loading_label["justify"] = "center"
        self.loading_label["text"] = ""
        self.loading_label.place(x=450,y=20,width=70,height=25)

    def select_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        if file_path:
            try:
                if not file_path.lower().endswith('.csv'):
                    raise ValueError("File is not a CSV")

                file_name = os.path.basename(file_path)  # Extract file name

                # Remove dataset.csv from src folder if it exists
                src_dataset_path = './src/dataset.csv'
                if os.path.exists(src_dataset_path):
                    os.remove(src_dataset_path)

                # Copy selected file to src folder and rename it to dataset.csv
                shutil.copy(file_path, './src/dataset.csv')
                messagebox.showinfo("File Updated", f"{file_name} copied and renamed as dataset.csv in src folder.")
                self.file_label.config(text=f"Selected CSV: {file_name}")  # Update label with file name
            except ValueError as e:
                messagebox.showerror("Invalid File", "Please select a valid CSV file.")
            except FileNotFoundError as e:
                messagebox.showerror("File Not Found", "The file 'dataset.csv' could not be found in the src folder.")
                
                
    def validate_batch_size(self, new_text):
        if new_text.isdigit() or new_text == "":
            return True
        return False

        
    def check_folder(self):
        folder_path = './src'
        if not os.path.exists(folder_path):
            messagebox.showerror("Folder Not Found", f"The folder '{folder_path}' does not exist!")
            self.master.destroy()
            
    def extract_image(self):
        # Check if necessary files and folders exist
        dataset_path = './src/dataset.csv'
        train_folder = './src/data/train'
        test_folder = './src/data/test'

        if not os.path.exists(dataset_path):
            messagebox.showerror("File Not Found", "The 'dataset.csv' file does not exist.")
            return
        else:
           
            self.loading_label.config(text=f"Loading...")    
            messagebox.showinfo("Loading", "Image extraction process...")
            time.sleep(2)
            if os.path.exists(train_folder) and os.path.exists(test_folder):
                # Remove the folders if they exist
                shutil.rmtree(train_folder)
                shutil.rmtree(test_folder)
            time.sleep(5)
            command = 'python src/dataset_prepare.py'
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, text=True)

            for line in process.stdout:
                # Process each line of the output here
                print(line.strip())

            # Wait for the process to finish and get the return code
            return_code = process.wait()

            if return_code == 0:
                print("Dataset preparation successful!")
                self.loading_label.config(text="Finish")
                messagebox.showinfo("Extraction Complete", "Image extraction process completed.")
            else:
                self.loading_label.config(text="Finish")
                print("Dataset preparation failed.")
                messagebox.showinfo("Extraction Failed", "Image extraction process failed.")
                
    
    def train_data(self):
        dataset_path = './src/dataset.csv'
        train_folder = './src/data/train'
        test_folder = './src/data/test'

        if not os.path.exists(dataset_path):
            messagebox.showerror("File Not Found", "The 'dataset.csv' file does not exist.")
        elif not (os.path.exists(train_folder) and os.path.exists(test_folder)):
            messagebox.showerror("Folders Not Found", "The 'data/train' or 'data/test' folders do not exist.")
        else:
            messagebox.showinfo("Loading", "Train Data process...")
            batch_size_value = self.entry_batch.get()
            epoch_value = self.entry_epoch.get()
            
            if not batch_size_value:
                batch_size_value = 64

            if not epoch_value:
                epoch_value = 50
            # Run emotions.py with specified epoch and batch_size
            
            command_train = 'python src/emotions.py --mode train --epoch {} --batch_size {}'.format(epoch_value, batch_size_value)
            process_train = subprocess.Popen(command_train, shell=True, stdout=subprocess.PIPE, text=True)

            for line in process_train.stdout:
                # Process each line of the output here
                print(line.strip())

            # Wait for the process to finish and get the return code
            return_code_train = process_train.wait()

            if return_code_train == 0:
                print("Train Data successful!")
                messagebox.showinfo("Train Data Complete", "Train Data process completed.")
            else:
                print("Train Data failed.")
                messagebox.showinfo("Train Data Failed", "Train Data process failed.")

    
    def model_default(self):
        default_model_path = './src/default/model.h5'  # Path to default model
        root_model_path = './src/model.h5'  # Root folder model path

        # Check if model exists in the default folder
        if os.path.exists(default_model_path):
            # Remove model from root folder if it exists
            if os.path.exists(root_model_path):
                os.remove(root_model_path)

            # Copy model from default folder to root folder
            shutil.copy(default_model_path, root_model_path)
            messagebox.showinfo("Model Updated", "Model updated from default model.")
        else:
            messagebox.showwarning("File Not Found", "file model.h5 not found in default folder.")
        

    def on_close_train_window(self):
        self.master.deiconify()  # Show the main window when the train window is closed

    def back_to_main(self, window):
        window.destroy()  # Close the train window
        self.master.deiconify()

def main():
    root = tk.Tk()
    root.geometry("600x600")
    root.resizable(False, False)
    app = ComputerVisionApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Training Data

94/448 [=====>........................] - ETA: 9:59 - loss: 1.8455 - accuracy: 0.2410


189/448 [===========>..................] - ETA: 7:31 - loss: 1.8313 - accuracy: 0.2469


284/448 [==================>...........] - ETA: 4:58 - loss: 1.8260 - accuracy: 0.2479


379/448 [========================>.....] - ETA: 2:04 - loss: 1.8174 - accuracy: 0.2507


448/448 [==============================] - 917s 2s/step - loss: 1.8050 - accuracy: 0.2578 - val_loss: 1.7214 - val_accuracy: 0.3329
1
Train Data successful!
Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.

1/1 [==============================] - 0s 333ms/step

1/1 [==============================] - 0s 42ms/step

1/1 [==============================] - 0s 42ms/step

1/1 [==============================] - 0s 57ms/step

1/1 [==============================] - 0s 83ms/step

1/1 [==============================] - 0s 51ms/step

1/1 [==============================] - 0s 58ms/step

1/1 [==============================] - 0s 53ms/step

1/1 [==============================] - 0s 58ms/step

1/1 [==============================] - 0s 58ms/step

1/1 [==============================] - 0s 48ms/step

1/1 [==============================] - 0s 48ms/step

1/1 [==============================] - 0s 62ms/step

1/1 [==============================] - 0s 62ms/step

1/1 [=========